In [2]:
import pandas as pd
from itertools import product

pd.options.display.max_columns = 100


def generate_test_vectors(n_inputs:int=0,labels:list[str]=[]):
    # labels[0] is MSB label
    if n_inputs:
        return pd.DataFrame(
            [*product(*((False, True) for _ in range(n_inputs)))],
            columns=[chr(65 + _) for _ in reversed(range(n_inputs))],
        )
    return pd.DataFrame(
        [*product(*((False, True) for _ in range(len(labels))))],
        columns=labels,
    )

def calc_cases(df: pd.DataFrame, checks: dict[str, pd.Series]):
    out = pd.DataFrame(index=df.index)
    for var, check in checks.items():
        tmp = df[var].where(check)
        out[f"{var}/gnd"] = tmp.where(tmp == True)
        out[f"{var}/vdd"] = tmp.where(tmp == False)
    out[out.notna()] = 1
    return out

# to select test cases:
# - select column with most ones and add it to test vector,
# - repeat, but skip rows, where testing is covered
# - if testing vestor is full, stop

In [3]:
# C17 circuit

df = generate_test_vectors(5)

# define/calculate intermediate signals in the structure
df["n1"] = ~(df.A & df.C)
df["n2"] = ~(df.D & df.C)
df["n3"] = ~(df.n2 & df.B)
df["n4"] = ~(df.n2 & df.E)

# define/calculate outputs
df["Y1"] = ~(df.n1 & df.n3)
df["Y2"] = ~(df.n3 & df.n4)

# define/calculate passthrough masks for signals
checks = {
    "A": (df.C & df.n3),
    "B": (df.n2 & (df.n1 | df.n4)),
    "C": (df.A & df.n3)
    | (df.D & df.B & (df.n1 | df.n4 | df.E))
    | (df.D & df.E & df.n3),
    "D": (df.C & df.B & (df.n1 | df.n4 | df.E)) | (df.C & df.E & df.n3),
    "E": (df.n2 & df.n3),
    "n1": df.n3,
    "n2": (df.D & (df.n1 | df.n4)) | (df.E & df.n3),
    "n3": df.n1 | df.n4,
    "n4": df.n3,
    "Y1": df.index >= 0,
    "Y2": df.index >= 0,
}

# calculate test vectors coverage for a SaF model
out_c17 = calc_cases(df, checks)
out_c17.fillna(0, inplace=True)
out_c17 = out_c17.T
out_c17.columns = pd.Index([str(i) for i in out_c17.columns])

C:\Users\PC\AppData\Local\Temp\ipykernel_1640\347113045.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out_c17.fillna(0, inplace=True)


In [4]:
out_c17

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
A/gnd,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
A/vdd,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
B/gnd,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0
B/vdd,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0
C/gnd,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1
C/vdd,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0
D/gnd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
D/vdd,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0
E/gnd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0
E/vdd,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# NO42 circuit

df = generate_test_vectors(4)

df["n3"] = ~(df.A | df.B)
df["n1"] = ~(df.D | df.C)
df["n2"] = ~(df.n1 & df.n3)

df["Q"] = ~(df.n2)

checks = {
    "A": (~df.B & df.n1),
    "B": (~df.A & df.n1),
    "C": (~df.D & df.n3),
    "D": (~df.C & df.n3),
    "n1": df.n3,
    "n2": df.index >= 0,
    "n3": df.n1,
    "Q": df.index >= 0,
}

out_no42 = calc_cases(df, checks)
out_no42.fillna(0, inplace=True)
out_no42 = out_no42.T
out_no42.columns = pd.Index([str(i) for i in out_no42.columns])

C:\Users\PC\AppData\Local\Temp\ipykernel_1640\15007426.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out_no42.fillna(0, inplace=True)


In [6]:
out_no42

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
A/gnd,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
B/gnd,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
B/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C/gnd,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
C/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
D/gnd,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
D/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
n1/gnd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
n1/vdd,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0


In [7]:
out_c17_docs = pd.read_csv("C17.csv", sep=';', index_col=0)
out_no42_docs = pd.read_csv("NO42.csv", sep=';', index_col=0)

In [8]:
out_c17.isin(out_c17_docs).T.all()

A/gnd      True
A/vdd      True
B/gnd      True
B/vdd      True
C/gnd      True
C/vdd      True
D/gnd      True
D/vdd      True
E/gnd      True
E/vdd      True
n1/gnd     True
n1/vdd     True
n2/gnd    False
n2/vdd    False
n3/gnd     True
n3/vdd     True
n4/gnd     True
n4/vdd     True
Y1/gnd     True
Y1/vdd     True
Y2/gnd     True
Y2/vdd     True
dtype: bool

In [9]:
out_no42.isin(out_no42_docs).T.all()

A/gnd     True
A/vdd     True
B/gnd     True
B/vdd     True
C/gnd     True
C/vdd     True
D/gnd     True
D/vdd     True
n1/gnd    True
n1/vdd    True
n2/gnd    True
n2/vdd    True
n3/gnd    True
n3/vdd    True
Q/gnd     True
Q/vdd     True
dtype: bool